<a href="https://colab.research.google.com/github/dan-a-iancu/OIT248/blob/main/BlueSky_RM/BlueSky_Airlines_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Blue Sky Airlines mini-case

____
# Basic Setup

Import useful modules, read the data and store it in data frames, and set up some useful Python lists. You may want to expand this section and make sure you understand how the data is organized, and also read the last part where the Python lists are created, as these may be very useful when you build your model.

In [18]:
#@markdown We first import some useful modules.

# import numpy
import numpy as np
import urllib.request  # for file downloading

# Import pandas for data-frames
import pandas as pd
pd.options.display.max_rows = 15
pd.options.display.float_format = "{:,.2f}".format

# Make sure Matplotlib runs inline, for nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import matplotlib.ticker as ticker

# install Gurobi (our linear optimization solver)
!pip install -i https://pypi.gurobi.com gurobipy
from gurobipy import *

print("Completed successfully!")

Looking in indexes: https://pypi.gurobi.com
Completed successfully!


## Load the case data into Pandas data frames

We first download an Excel file with all the data from Github.

In [19]:
#@markdown Download the entire data as an Excel file from Github

url_Excel = 'https://github.com/dan-a-iancu/airm/blob/master/BlueSky_RM/BlueSky_Data.xlsx?raw=true'
local_file = "BlueSky_Data.xlsx"   # name of local file where you want to store the downloaded file
urllib.request.urlretrieve(url_Excel, local_file)    # download from website and save it locally

('BlueSky_Data.xlsx', <http.client.HTTPMessage at 0x7a8eeaa732e0>)

Read in and store the data in suitable dataframes.

In [22]:
# Create dataframes based on the Excel file

# Read in all the information about the itinerarys sold, and the aircraft capacities
itineraryData = pd.read_excel("BlueSky_Data.xlsx", sheet_name = "Itinerary_data", index_col=0)
print("DataFrame with information on itineraries/tickets:")
display(itineraryData)

aircraftData = pd.read_excel("BlueSky_Data.xlsx", sheet_name = "Aircraft_data", index_col=0)
print("DataFrame with information on flight legs:")
display(aircraftData)

DataFrame with information on itineraries/tickets:


,Q_Fare,Y_Fare,Q_Demand,Y_Demand,BOS_ORD_Flight,JFK_ORD_Flight,ORD_SFO_Flight,ORD_LAX_Flight
BOS_ORD,200,220,25,20,1,0,0,0
BOS_SFO,320,420,55,40,1,0,1,0
BOS_LAX,400,490,65,25,1,0,0,1
JFK_ORD,250,290,24,16,0,1,0,0
JFK_SFO,410,540,65,50,0,1,1,0
JFK_LAX,450,550,40,35,0,1,0,1
ORD_SFO,210,230,21,50,0,0,1,0
ORD_LAX,260,300,25,14,0,0,0,1


DataFrame with information on flight legs:


,Capacity
BOS_ORD_Flight,200
JFK_ORD_Flight,200
ORD_SFO_Flight,200
ORD_LAX_Flight,200


## Create Python lists based on the data-frames

__NOTE__: Make sure you understand what the __lists__ created here are! These will be very helpful when creating the model.

In [4]:
# Create useful lists
numODPairs = len(itineraryData)               # number of origin-destination (O-D) pairs used for itinerarys
allODPairs = list(itineraryData.index)        # a list with all the O-D pairs (rows in itineraryData)
numFlights = len(aircraftData)             # number of distinct flights/legs operated by the Airline
allFlights = list(aircraftData.index)      # a list with all the flights operated (rows in aircraftData)

# have a look
print(allODPairs)
print(allFlights)

['BOS_ORD', 'BOS_SFO', 'BOS_LAX', 'JFK_ORD', 'JFK_SFO', 'JFK_LAX', 'ORD_SFO', 'ORD_LAX']
['BOS_ORD_Flight', 'JFK_ORD_Flight', 'ORD_SFO_Flight', 'ORD_LAX_Flight']


<font color=red>**IMPORTANT HINT.**</font> When defining your **decision variables** in this model, you may want to consider *matrices* (or grids) of decision variables instead of a simple list like we've created before. Specifically, you may want to add a decision variable for every pair of elements where the first element is from one list and the second element is from another list.

When the decision variables are organized in a grid, it's significantly easier to use the ``addVars`` function to add all of the decision variables to the model at once. The function ``addVars`` can take as arguments several lists, e.g., `addVars(list1, list2, list3,...)`, in which case it will return one decision variable for every possible tuple of values. These decision variables will be stored as a dictionary, with the dictionary keys corresponding to the tuple of values.

For instance, suppose `list1 = ["Paris", "Budapest", "Beijing"]` and `list2= [1,2]`. Then `myDecisions = addVars(list1,list2)` would return a dictionary containing one decision for every pair, i.e., ("Paris",1), ("Paris",2), ("Budapest",1), ("Budapest",2), etc. To access the decision corresponding to the pair ("Paris",1), we can simply use `myDecisions[("Paris",1)]` or `myDecisions["Paris",1]`.

## Read data

In [5]:
# Read in all the information about the itinerarys sold, and the aircraft capacities
itineraryData = pd.read_excel("BlueSky_Data.xlsx", sheet_name = "Itinerary_data", index_col=0)
aircraftData = pd.read_excel("BlueSky_Data.xlsx", sheet_name = "Aircraft_data", index_col=0)

In [6]:
# Have a look at the data frames:
display(itineraryData)
display(aircraftData)

,Q_Fare,Y_Fare,Q_Demand,Y_Demand,BOS_ORD_Flight,JFK_ORD_Flight,ORD_SFO_Flight,ORD_LAX_Flight
BOS_ORD,200,220,25,20,1,0,0,0
BOS_SFO,320,420,55,40,1,0,1,0
BOS_LAX,400,490,65,25,1,0,0,1
JFK_ORD,250,290,24,16,0,1,0,0
JFK_SFO,410,540,65,50,0,1,1,0
JFK_LAX,450,550,40,35,0,1,0,1
ORD_SFO,210,230,21,50,0,0,1,0
ORD_LAX,260,300,25,14,0,0,0,1


,Capacity
BOS_ORD_Flight,200
JFK_ORD_Flight,200
ORD_SFO_Flight,200
ORD_LAX_Flight,200


<font color=blue>**HINT!** Have a close look at the `itineraryData` DataFrame above. Note that the columns `BOS_ORD_Flight`, `JFK_ORD_Flight`, etc. record whether or not a **seat** is needed for that flight by the itinerary on the row. For instance, the **second row** is for an itinerary **BOS_SFO** (in either Q-class or Y-class), and note that this requires one seat on the **BOS_ORD** flight and one seat on the **ORD_SFO** flight, so that the columns correspoding to these flights (`BOS_ORD_Flight` and `ORD_SFO_Flight`) both have an entry of 1. These columns containing 1 and 0 may be very useful when formulating some of your constraints!</blue>

## Calculate some useful values and lists

In [7]:
# a list with all the itineraries (i.e., origin-destination pairs), corresponding to tickets sold
all_itineraries = list(itineraryData.index)

# a list with all the flight legs operated (physical flights)
all_flights = list(aircraftData.index)

In [8]:
# have a look
print(all_itineraries)
print(all_flights)

['BOS_ORD', 'BOS_SFO', 'BOS_LAX', 'JFK_ORD', 'JFK_SFO', 'JFK_LAX', 'ORD_SFO', 'ORD_LAX']
['BOS_ORD_Flight', 'JFK_ORD_Flight', 'ORD_SFO_Flight', 'ORD_LAX_Flight']


# **Q1.**
## Model

In [9]:
m = Model("Blue Sky Airline Model")

Restricted license - for non-production use only - expires 2024-10-28


## Decision Variables
We have one decision for every possible "ticket type" sold, i.e., every origin-destination-fare class. We can denote these by $Q[i]$ and $Y[i]$ respectively, for the number of tickets sold in Q-class and Y-class, respectively, on the itinerary $i$, where $i$ belongs to the list of itineraries, ['BOS_ORD', 'BOS_SFO', 'BOS_LAX', 'JFK_ORD', 'JFK_SFO', 'JFK_LAX', 'ORD_SFO', 'ORD_LAX']

In [10]:
# one decision for each booking limit, i.e., how many itinerarys of each type to sell (these are 'sales')
Q_tickets_sold = m.addVars( all_itineraries, name ="Q_sales" )
Y_tickets_sold = m.addVars( all_itineraries, name ="Y_sales" )

## Objective
The objective is to maximize total revenue.

In [11]:
# revenues are given by sales of all the itinerarys at correct fares
revenue = quicksum( Q_tickets_sold[i]*itineraryData.loc[i,"Q_Fare"] for i in all_itineraries ) + \
quicksum( Y_tickets_sold[i]*itineraryData.loc[i,"Y_Fare"] for i in all_itineraries )

# set the objective
m.setObjective(revenue, GRB.MAXIMIZE)

## Constraints
The first set of constraints is that the tickets sold on a specific itinerary cannot exceed the demand for that itinerary. Mathematically:
\begin{align}
  Q[i] \leq \text{Q_demand}[i] \\
  Y[i] \leq \text{Y_demand}[i]
\end{align}
where $\text{Q_demand}[i]$ and $\text{Y_demand}[i]$ correspond to the Q-class and Y-class demand for tickets given in the itinerary data.

In [12]:
# demand constraints: cannot sell more than the demand for each itinerary - fare class
demand_constraints = {}
for i in all_itineraries:
    # Q-class
    cons_name = "itineraries_" + i + "_Q"
    demand_constraints[cons_name] = m.addConstr( Q_tickets_sold[i] <= itineraryData.loc[i,"Q_Demand"], \
                                                name = cons_name)

    # Y-class
    cons_name = "itineraries_" + i + "_Y"
    demand_constraints[cons_name] = m.addConstr( Y_tickets_sold[i] <= itineraryData.loc[i,"Y_Demand"], \
                                                name = cons_name )

The second set of constraints correspond to having enough seats on the flights to accomodate all the passengers with tickets. We have 4 flights, and we need to write one constraint for each flight, i.e., for each element of the list `all_flights`. For a given flight, we need to add all the tickets/itineraries that utilize that flight, and the total sum must not exceed the plane capacity.

One option is to do the summation in a very careful way. For instance, for the flight **BOS-ORD**, we would need to add up all the itineraries that use this flight, i.e., **BOS-ORD**, **BOS-SFO**, **BOS-LAX**, in both Q-class and Y-class.

A second option (far simpler!!) is to use the columns `BOS_ORD_Flight`,	`JFK_ORD_Flight`, `ORD_SFO_Flight`, `ORD_LAX_Flight` from the `itineraryData` DataFrame, which contain 1 for the rows/itineraries that use the flight and 0 otherwise. With this, we can simply add over all the rows/itineraries and multiply with these 0/1 indicators, which allows us to sum up only the terms/tickets that actually use up the respective flight.

In [13]:
# capacity constraints: one for each individual flight operated
flight_constraints = {}
for f in all_flights :
    # f denotes a particular flight, i.e., a row in 'aircraftData'

    # calculate how many itineraries were sold that require seats on that flight
    # NOTE. Here, we just add over ALL itineraries, but we multiply with the 0/1 indicators
    m.addConstr( quicksum( (Q_tickets_sold[i] + Y_tickets_sold[i])*itineraryData.loc[i,f] \
                          for i in all_itineraries ) \
                           <= aircraftData.loc[f,"Capacity"], name = f )

## Print out and inspect the model

In [14]:
m.write("BlueSky.lp")  # write model information to an .ilp file
f = open("BlueSky.lp", 'r')    # open a file handle
print( f.read() )           # read the contents and print them
f.close()              # close the file handle

\ Model Blue Sky Airline Model
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  200 Q_sales[BOS_ORD] + 320 Q_sales[BOS_SFO] + 400 Q_sales[BOS_LAX]
   + 250 Q_sales[JFK_ORD] + 410 Q_sales[JFK_SFO] + 450 Q_sales[JFK_LAX]
   + 210 Q_sales[ORD_SFO] + 260 Q_sales[ORD_LAX] + 220 Y_sales[BOS_ORD]
   + 420 Y_sales[BOS_SFO] + 490 Y_sales[BOS_LAX] + 290 Y_sales[JFK_ORD]
   + 540 Y_sales[JFK_SFO] + 550 Y_sales[JFK_LAX] + 230 Y_sales[ORD_SFO]
   + 300 Y_sales[ORD_LAX]
Subject To
 itineraries_BOS_ORD_Q: Q_sales[BOS_ORD] <= 25
 itineraries_BOS_ORD_Y: Y_sales[BOS_ORD] <= 20
 itineraries_BOS_SFO_Q: Q_sales[BOS_SFO] <= 55
 itineraries_BOS_SFO_Y: Y_sales[BOS_SFO] <= 40
 itineraries_BOS_LAX_Q: Q_sales[BOS_LAX] <= 65
 itineraries_BOS_LAX_Y: Y_sales[BOS_LAX] <= 25
 itineraries_JFK_ORD_Q: Q_sales[JFK_ORD] <= 24
 itineraries_JFK_ORD_Y: Y_sales[JFK_ORD] <= 16
 itineraries_JFK_SFO_Q: Q_sales[JFK_SFO] <= 65
 itineraries_JFK_SFO_Y: Y_sales[JFK_SFO] <= 50
 itineraries_JFK_

## Solve the model

In [15]:
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 20 rows, 16 columns and 40 nonzeros
Model fingerprint: 0xcc49b0a9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 16 rows and 0 columns
Presolve time: 0.02s
Presolved: 4 rows, 16 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3461000e+05   7.237500e+01   0.000000e+00      0s
       7    1.8285000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.828500000e+05


## Print the solution

In [16]:
for i in all_itineraries:
    print(f"Route {i:<15s} Q-class: {Q_tickets_sold[i].X:.2f} \t  Y-class: {Y_tickets_sold[i].X:.2f}")

Route BOS_ORD         Q-class: 25.00 	  Y-class: 20.00
Route BOS_SFO         Q-class: 25.00 	  Y-class: 40.00
Route BOS_LAX         Q-class: 65.00 	  Y-class: 25.00
Route JFK_ORD         Q-class: 24.00 	  Y-class: 16.00
Route JFK_SFO         Q-class: 35.00 	  Y-class: 50.00
Route JFK_LAX         Q-class: 40.00 	  Y-class: 35.00
Route ORD_SFO         Q-class: 0.00 	  Y-class: 50.00
Route ORD_LAX         Q-class: 21.00 	  Y-class: 14.00


# **Q2.**
Let's extract the information on all the constraints.

In [17]:
# display information on all the constraints
allConstraints = m.getConstrs()

all_constr_data = [ [c.ConstrName, m.getRow(c).getValue(), c.Sense, c.RHS, c.Slack, \
                     c.Pi, c.SARHSLow, c.SARHSUp ] for c in allConstraints ]

# convert into a dataframe and print
df = pd.DataFrame(all_constr_data, columns=["Constraint Name", "LHS Value", "Sense", "RHS Value", "Slack", "Shadow Price",\
                                           "Lower Limit RHS", "Upper Limit RHS"])
display(df)

,Constraint Name,LHS Value,Sense,RHS Value,Slack,Shadow Price,Lower Limit RHS,Upper Limit RHS
0,itineraries_BOS_ORD_Q,25.00,<,25.00,0.00,100.00,25.00,29.00
1,itineraries_BOS_ORD_Y,20.00,<,20.00,0.00,120.00,20.00,24.00
2,itineraries_BOS_SFO_Q,25.00,<,55.00,30.00,0.00,25.00,inf
3,itineraries_BOS_SFO_Y,40.00,<,40.00,0.00,100.00,10.00,65.00
4,itineraries_BOS_LAX_Q,65.00,<,65.00,0.00,40.00,65.00,90.00
...,...,...,...,...,...,...,...,...
15,itineraries_ORD_LAX_Y,14.00,<,14.00,0.00,40.00,10.00,35.00
16,BOS_ORD_Flight,200.00,<,200.00,0.00,100.00,196.00,200.00
17,JFK_ORD_Flight,200.00,<,200.00,0.00,190.00,196.00,200.00
18,ORD_SFO_Flight,200.00,<,200.00,0.00,220.00,200.00,204.00
